In [3]:

"""Code Written from Scratch by Bivek Sapkota"""
"""Generates single but random set of Prompt and Response at each run"""
no_of_dataset= 3000 #this no. of dataset will be generated as prompt and response pair

import datetime
import random
now = datetime.datetime.now()

code= []
code.append("%\n") #header

#generating FileName
random_number =random.randint(0,9)
gcodeFileName= now.strftime("O%M%S")+str(random_number) # generates filename with O at front and minutes and seconds with single digit random number
code.append(f"{gcodeFileName} (Gcode for milling rectangular stock with chamfering/rounding) ;\n")

#startup Sequence
#Safe Startup
#G90 absolute coordinate system
#G80 cancels all canned cycles
#G40 turns off cutter compensation
#G49 cancels tool length compensation
#G54 activates coordinate systems, custom home position, work coordinate is set to top left corner
code.append("G90 G80 G40 G49 G54 (Safe startup) ;\n")

# Define tool change
toolLocation= random.choice(["T1","T2","T3","T4","T5","T6"])
code.append(f"{toolLocation} M06 (Select tool {toolLocation});\n")

#Startup Sequence
# code.append("G28 (Home all axes);\n")
#randomizing unit type
# unitType= random.randint(0,1)
# if unitType==0:
#   code.append("G54 (mm units);\n")
# else:
#   code.append("G53 (inch units);\n")

#Tool Length Compensation
toolLengthLocation = random.choice(["H01","H02","H03","H04","H05"])
code.append(f"G43 {toolLengthLocation} (Activate tool length offset {toolLengthLocation});\n")

#Tool Diameter
toolDiameterLocation= random.choice(["D01","D02","D03","D04","D05"])
def toolDiameter():
  if toolDiameterLocation== "D01":
    toolDiameter= 5
  if toolDiameterLocation== "D02":
    toolDiameter= 10
  if toolDiameterLocation== "D03":
    toolDiameter= 15
  if toolDiameterLocation== "D04":
    toolDiameter= 20
  if toolDiameterLocation== "D05":
    toolDiameter= 25
  return toolDiameter

code.append(f"G42 {toolDiameterLocation} (Set tool diameter to {toolDiameter()}mm);\n")

#define material and RPM

def rpm(m):
  RPM= 1000 #default value
  if m== "aluminium":
    RPM= 2200
  if m== "copper":
    RPM= 1500
  if m== "steel":
    RPM= 1200
  if m== "plastic":
    RPM= 8000
  return RPM

material= random.choice(["aluminum","copper","steel", "plastic"])

#randomize spindle RPM
spindleRPM= rpm(material)
code.append(f"S{spindleRPM} M03 (Rotate spindle {spindleRPM}RPM Clockwise);\n")


#Deciding to turn on Coolant
if material == "aluminum" or material == "copper" or material == "steel":
  coolantflag=1
else:
  coolantflag=0

if coolantflag == 1:
  code.append("M08 (Coolant on) ;\n")

#define FeedRate depending upon material
def feedRate(m):
  feedRate= 1000 #default value"
  if m== "aluminium":
    feedRate= 1500
  if m== "copper":
    feedRate= 800
  if m== "steel":
    feedRate= 500
  if m== "plastic":
    feedRate= 3000
  return feedRate

#define Dimension of stock
length= random.randint(2,20)*10
width= random.randint(2,20)*10
height= random.randint(1,40)
bottomSurfaceDistanceFromTable= random.randint(5,50)
additionalEngagement= random.randint(5,10)
sidetoothEngagement= height+ additionalEngagement

topSurfaceDistanceFromTable= height+ bottomSurfaceDistanceFromTable
x_start= 0
y_start= 0
z_start= topSurfaceDistanceFromTable

#Rapid move to coordinate system

#now lets start cutting
#cutting started from the side of stock

#Rapid move to work coordinate
code.append(f"G00 G54 (Rapid to work coordinate);\n")

#initialize cut
extraLength= height*0.25
cuttingDepth= height+ extraLength
code.append(f"G01 Z{cuttingDepth} F{feedRate(material)}. (Feed to cutting depth) ;\n") #starts cutting at the depth of
# start cutting
# cutting a length x width stock, the top left corner is at x_start, y_start, cutting starts from top left corner, cutting progresses anticlockwise
#randomize if there should be radius or chamfering at the corner

toolDiameter= random.randint(2,20)

def radiusOrChamferSyntax():
  radiusOrChamferorNull= random.choice([0,1,2])
  if radiusOrChamferorNull==0:
    syntax= ",R"
    radiusValue= random.randint(0,toolDiameter) #setting radius value to max of milltool diameter
    syntax= syntax+str(radiusValue)
  elif radiusOrChamferorNull==1:
    syntax= ",C"
    chamferValue= random.randint(0,toolDiameter) #setting chamfer value to max of milltool diameter
    syntax= syntax+str(chamferValue)
  else:
    syntax= ""
  return syntax

#G54 is set for upper left corner
currentPosition= "topLeftStart"
code.append(f"X{x_start} Y{y_start} (at top left initial position);\n")

bottomLeftSyntax= radiusOrChamferSyntax()
code.append(f"X{x_start} Y{y_start- width} {bottomLeftSyntax} (at bottom left position);\n")
bottomLeftProcessText= f"Chamfer of {bottomLeftSyntax.split(',C')[1]}" if ",C" in bottomLeftSyntax else (f"Radius of {bottomLeftSyntax.split(',R')[1]}" if ",R" in bottomLeftSyntax else "")
bottomLeftWholetext= f"The bottom left corner needs a {bottomLeftProcessText}mm." if not bottomLeftProcessText == "" else ""

bottomRightSyntax= radiusOrChamferSyntax()
code.append(f"X{x_start+ length} Y{y_start-width} {bottomRightSyntax} (at bottom right position);\n")
bottomRightProcessText= f"Chamfer of {bottomRightSyntax.split(',C')[1]}" if ",C" in bottomRightSyntax else (f"Radius of {bottomRightSyntax.split(',R')[1]}" if ",R" in bottomRightSyntax else "")
bottomRightWholetext= f"The bottom right corner needs a {bottomRightProcessText}mm." if not bottomRightProcessText == "" else ""

topRightSyntax= radiusOrChamferSyntax()
code.append(f"X{x_start+ length} Y{y_start} {topRightSyntax} (at top right position);\n")
topRightProcessText= f"Chamfer of {topRightSyntax.split(',C')[1]}" if ",C" in topRightSyntax else (f"Radius of {topRightSyntax.split(',R')[1]}" if ",R" in topRightSyntax else "")
topRightWholetext= f"The top right corner needs a {topRightProcessText}mm." if not topRightProcessText == "" else ""

topLeftSyntax= radiusOrChamferSyntax()
code.append(f"X{x_start} Y{y_start} {topLeftSyntax} (at initial top left position);\n")
topLeftProcessText= f"Chamfer of {topLeftSyntax.split(',C')[1]}" if ",C" in topLeftSyntax else (f"Radius of {topLeftSyntax.split(',R')[1]}" if ",R" in topLeftSyntax else "")
topLeftWholetext= f"The top left corner needs a {topRightProcessText}mm." if not topRightProcessText == "" else ""

#completion sequence
#tool Retraction
code.append(f"G00 Z{z_start+0.1} (Rapid Retract of Tool);\n")

#Spindle Off
code.append("M05 (spindle off);\n") #turn spindle off

#coolant off
if coolantflag == 1:
  code.append("M09 (Coolant off) ;\n")
# code.append("G53 X0 Y0 (X and Y home);\n")
code.append("M30 (End Program);\n")
code.append("%")

prompt= f"A rectangular {material} block of length {length}mm, width {width}mm and a height of {height}mm is fixtured in a milling machine, \
where the stock has to be milled on all 4 sides by {0}mm. \
{topLeftWholetext} {topRightWholetext} {bottomLeftWholetext} {bottomRightWholetext}\n\
A mill tool of diameter {toolDiameter}mm \
having a side tooth engagement of {sidetoothEngagement}mm which is greater than the thickness of stock is used for milling.\n\
The bottom tip of the tool will be positioned {extraLength}mm below the part surface. \nThe cutter will be operated at a spindle speed of {spindleRPM} RPM \
and at a feed rate of {feedRate(material)} mm/min.\nThe TM1 mill cutter is in the location {toolLocation} \
with tool length stored in {toolLengthLocation}, tool diameter stored in {toolDiameterLocation}"

# Print the prompt 
print("Prompt\n")
print(prompt)
print("\n")

# Appends line number in Nxx notation
finalcode= [] #all the lines of gcodes are stored in this list
for index, mygcodes in enumerate(code):
  if index<2 or index== len(code)-1:
    finalcode.append(mygcodes)
  else:
    finalcode.append(f"N{index-1} {mygcodes}")

#Prints the Response (GCODE)
for i in finalcode:
  print(i, end="")


Prompt

A rectangular copper block of length 110mm, width 20mm and a height of 33mm is fixtured in a milling machine, where the stock has to be milled on all 4 sides by 0mm. The top left corner needs a Chamfer of 7mm. The top right corner needs a Chamfer of 7mm.  
A mill tool of diameter 12mm having a side tooth engagement of 41mm which is greater than the thickness of stock is used for milling.
The bottom tip of the tool will be positioned 8.25mm below the part surface. 
The cutter will be operated at a spindle speed of 1500 RPM and at a feed rate of 800 mm/min.
The TM1 mill cutter is in the location T1 with tool length stored in H04, tool diameter stored in D02


%
O13161 (Gcode for milling rectangular stock with chamfering/rounding) ;
N1 G90 G80 G40 G49 G54 (Safe startup) ;
N2 T1 M06 (Select tool T1);
N3 G43 H04 (Activate tool length offset H04);
N4 G42 D02 (Set tool diameter to 10mm);
N5 S1500 M03 (Rotate spindle 1500RPM Clockwise);
N6 M08 (Coolant on) ;
N7 G00 G54 (Rapid to work c